 #  <p style="text-align: center;"> Recomendação de itens aos usuários</p>

A Filtragem Colaborativa é uma técnica popular usada para criar recomendações de Usuário-Item. A lista utilizada será de quais usuários compraram quais itens e criaremos uma pontuação de afinidade entre os itens. Quando um cliente compra um item, usamos essa pontuação de afinidade para recomendar outros itens a ele.

## Carregando o conjunto de dados

O conjunto de dados de origem contém em cada fileira, uma ID de usuário e uma ID de item. Esta é a lista de usuários e os itens que eles compraram, uma linha por vez

In [ ]:
import pandas as pd

userItemData = pd.read_csv('03_05_ratings.csv')
userItemData.head()

,usuarioId,ItemId
0,1001,5001
1,1001,5002
2,1001,5005
3,1002,5003
4,1002,5004


## Criando uma pontuação de afinidade

Criando um algoritmo para gerar afinidades de usuário/item.

In [ ]:
#Obter a lista de itens únicos
itemList=list(set(userItemData["ItemId"].tolist()))

#Obter a contagem de usuários
userCount=len(set(userItemData["usuarioId"].tolist()))

#Criar um data frame vazio para armazenar a pontuação de afinidade por itens.
itemAffinity= pd.DataFrame(columns=('item1', 'item2', 'pontuacao'))
rowCount=0

#Para cada item da lista, comparar com outros itens.
for ind1 in range(len(itemList)):

    #Obter a lista de usuários que compraram o item 1.
    item1Users = userItemData[userItemData.ItemId==itemList[ind1]]["usuarioId"].tolist()
    #print("Item 1 ", item1Users)

    #Obter os itens que não são o item 1 ou que não foram analisados ainda.
    for ind2 in range(ind1, len(itemList)):

        if ( ind1 == ind2):
            continue

        #Obter a lista de usuários que comrparam o item 2
        item2Users=userItemData[userItemData.ItemId==itemList[ind2]]["usuarioId"].tolist()
        #print("Item 2",item2Users)

        #Encontrar pontuação. Encontrar a lista de usuários em comum e dividi-la pelo total de usuários.
        commonUsers= len(set(item1Users).intersection(set(item2Users)))
        pontuacao=commonUsers / userCount

        #Adicionar pontuação para o item 1, item 2
        itemAffinity.loc[rowCount] = [itemList[ind1],itemList[ind2],pontuacao]
        rowCount +=1
        #Adicionar pontuação para o item 1, item 2. A mesma pontuação se aplica independente da sequência.
        itemAffinity.loc[rowCount] = [itemList[ind2],itemList[ind1],pontuacao]
        rowCount +=1

#Checar resultado final
itemAffinity.head()


,item1,item2,pontuacao
0,5001.0,5002.0,0.4
1,5002.0,5001.0,0.4
2,5001.0,5003.0,0.0
3,5003.0,5001.0,0.0
4,5001.0,5004.0,0.2


## Recomendação de itens

Considerando que o cliente comprou um item 5001. Podemos consultar esse dataframe para o item1-5001 e obter itens2 com pontuação em ordem decrescente. Esta é a lista de itens que vocês podem recomendar ao usuário nessa ordem. Esta análise em um DF com muitos itens, pode ser realizado também por meio de um loop.

In [ ]:
searchItem=5001
recoList=itemAffinity[itemAffinity.item1==searchItem]\
        [["item2","pontuacao"]]\
        .sort_values("pontuacao", ascending=[0])

print("Recomendações para o item 5001\n", recoList)

Recomendações para o item 5001
     item2  pontuacao
0  5002.0        0.4
6  5005.0        0.4
4  5004.0        0.2
2  5003.0        0.0
